In [170]:
import torch
import torch.nn as nn
from torch.autograd import Variable

import numpy as np
import string
from tqdm import tqdm_notebook

all_letters = string.ascii_letters + " .,;'"

def read_data():
    data_path = 'data/HPserie/HPSerie'
    with open(data_path, 'r') as myfile:
        data=myfile.read().replace('\n', '')
    return data

def seq_gen(data, data_enc, seq_len, batch_size):
    total_size = len(data)
    ex_len = seq_len + 1
    idxs = list(range(0, total_size-200, ex_len))
    np.random.shuffle(idxs)
    
    yield len(idxs) // batch_size 
    while True:
        batches = np.array_split(idxs, len(idxs) // batch_size)    
        for batch in batches:
            X = [data[i:i+seq_len] for i in batch]
            Y = [data[i+1:i+1+seq_len] for i in batch]
            X_enc = torch.Tensor([data_enc[i:i+seq_len] for i in batch]).t().long().cuda()
            Y_enc = torch.Tensor([data_enc[i+1:i+1+seq_len] for i in batch]).t().long().cuda()            
             
            yield Variable(X_enc), Variable(Y_enc)
        np.random.shuffle(idxs)
    

data = read_data()
data = [x for x in data if x in all_letters]

data_enc = [char_to_idx[x] for x in data]

char_to_idx = {x: i for i,x in enumerate(all_letters)}
idx_to_char = {v: k for k,v in char_to_idx.items()}
G = seq_gen(data, data_enc, 200, 50)

In [171]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden, seq_len, batch_size):
        input = self.encoder(input)
        output, hidden = self.gru(input.view(seq_len, batch_size, -1), hidden)
        output = self.decoder(output.view(-1, self.hidden_size))
        output = output.view(seq_len, batch_size, self.output_size)
        return output, hidden

    def init_hidden(self, batch_size):
        return Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size).cuda())

In [172]:
input_size = len(all_letters)
hidden_size = 100
n_layers = 3

print_every = 50 
plot_every = 100
epochs = 10


model = RNN(input_size, hidden_size, input_size, n_layers)
model_optimizer = torch.optim.Adam(model.parameters())

model.cuda()

RNN (
  (encoder): Embedding(57, 100)
  (gru): GRU(100, 100, num_layers=3)
  (decoder): Linear (100 -> 57)
)

In [ ]:
criterion = nn.CrossEntropyLoss()

def train(inp, target):
    hidden = model.init_hidden(inp.size()[1])
    model.zero_grad()
    loss = 0

    output, hidden = model(inp, hidden, inp.size()[0], inp.size()[1])
    output = output.view(-1, model.output_size)
    loss = criterion(output, target.view(-1))

    loss.backward()
    model_optimizer.step()

    return loss.data[0]

In [ ]:
G = seq_gen(data, data_enc, 200, 100)
batch_per_epoch = next(G)

loss_avg = 0
losses = []

for epoch in range(1, epochs+1):
    bar = tqdm_notebook(range(batch_per_epoch))
    for b in bar:
        X_enc, Y_enc = next(G)
        loss = train(X_enc, Y_enc)       
        loss_avg += loss
        
        bar.set_postfix(loss=loss)
        if b == batch_per_epoch-1:
            bar.write(loss_avg / batch_per_epoch)
            loss_avg = 0

In [ ]:
torch.cuda.is_available()

In [51]:
X = np.zeros((10,4))
X = torch.Tensor(X).long().cuda()
X = Variable(X)

out = model(X, model.init_hidden())
out[0].size()

torch.Size([10, 4, 100])
torch.Size([10, 4, 100])


torch.Size([10, 4, 57])